In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'NLP/Final_Project/'

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
import os
os.environ["WANDB_MODE"] = "disabled"


In [3]:
! pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [4]:
from torch.utils.data import Dataset
import pandas as pd

class TextToTextDataset(Dataset):
    def __init__(self, csv_file, tokenizer):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]['Input_text']
        output_text = self.data.iloc[idx]['Output_text']

        input_tokens = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        output_tokens = self.tokenizer(output_text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

        return {
            'input_ids': input_tokens['input_ids'].squeeze(),
            'attention_mask': input_tokens['attention_mask'].squeeze(),
            'labels': output_tokens['input_ids'].squeeze()
        }

In [5]:
def compute_accuracies(model, tokenizer, eval_dataset, batch_size=16):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    dataloader = DataLoader(eval_dataset, batch_size=batch_size)
    sequence_correct = 0
    total_positions = 0
    position_correct = 0
    total_sequences = 0

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels']

            outputs = model.generate(
                input_ids,
                max_length=128,
                pad_token_id=tokenizer.pad_token_id
            )

            predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
            true_labels = [tokenizer.decode(label[label != -100], skip_special_tokens=True) for label in labels]

            for pred, true in zip(predictions, true_labels):
                # Split by semicolon and strip whitespace
                pred_colors = [color.strip() for color in pred.split(';')]
                true_colors = [color.strip() for color in true.split(';')]

                if pred_colors == true_colors:
                    sequence_correct += 1
                total_sequences += 1

                min_len = min(len(pred_colors), len(true_colors))
                for i in range(min_len):
                    if pred_colors[i] == true_colors[i]:
                        position_correct += 1
                total_positions += len(true_colors)

    sequence_accuracy = sequence_correct / total_sequences
    position_accuracy = position_correct / total_positions

    return {
        'sequence_accuracy': sequence_accuracy,
        'position_accuracy': position_accuracy,
        'total_sequences': total_sequences,
        'total_positions': total_positions
    }

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

model_checkpoint1 = "t5-small"
tokenizer1 = AutoTokenizer.from_pretrained(model_checkpoint1)
model1 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
test_dataset1 = TextToTextDataset("/content/drive/My Drive/NLP/Final_Project/test/navigation_dataset_Test.csv", tokenizer1)

data_collator1 = DataCollatorForSeq2Seq(tokenizer1, model=model1)

In [8]:
results = compute_accuracies(model1, tokenizer1, test_dataset1)
print(f"Sequence Accuracy: {results['sequence_accuracy']:.4f}")
print(f"Position Accuracy: {results['position_accuracy']:.4f}")
print(f"Total sequences evaluated: {results['total_sequences']}")
print(f"Total positions evaluated: {results['total_positions']}")

100%|██████████| 299/299 [05:23<00:00,  1.08s/it]

Sequence Accuracy: 0.0000
Position Accuracy: 0.0000
Total sequences evaluated: 4781
Total positions evaluated: 19430


In [9]:
model_path2 = "/content/drive/My Drive/NLP/Final_Project/test/ft_model"
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_path2)
tokenizer2 = AutoTokenizer.from_pretrained(model_path2)

In [10]:
test_dataset2 = TextToTextDataset("/content/drive/My Drive/NLP/Final_Project/test/navigation_dataset_Test.csv", tokenizer2)

data_collator2 = DataCollatorForSeq2Seq(tokenizer2, model=model2)

In [11]:
results = compute_accuracies(model2, tokenizer2, test_dataset2)
print(f"Sequence Accuracy: {results['sequence_accuracy']:.4f}")
print(f"Position Accuracy: {results['position_accuracy']:.4f}")
print(f"Total sequences evaluated: {results['total_sequences']}")
print(f"Total positions evaluated: {results['total_positions']}")

100%|██████████| 299/299 [00:45<00:00,  6.55it/s]

Sequence Accuracy: 1.0000
Position Accuracy: 1.0000
Total sequences evaluated: 4781
Total positions evaluated: 19430
